In [1]:
import cymetric as cym
from cymetric import filters 
from cymetric import graphs 
from cymetric import timeseries 
file = '../cyclus.sqlite'
file = cym.tools.dbopen(file)
ev = cym.Evaluator(file, write=False)

In [ ]:
graphs.flow_graph(ev, commodities=[''])

KeyError: 'Senders'

In [18]:
set(ev.eval('Transactions')['Commodity'])

{'fresh_uox', 'spent_uox', 'tails', 'u_ore'}

In [12]:
ev.metrics.keys()

dict_keys(['Materials', 'Resources', 'Compositions', 'Transactions', 'AgentEntry', 'Agents', 'AgentExit', 'DecomSchedule', 'Info'])